In [1]:
# [3 marks] Write codes to expose a Flask API on port 80 running in a Docker container.

from flask import Flask, flash, request, redirect, url_for, Response
import requests
import os
import json

# Configure our application -using local url / port 80

tensorflow_server1_url = "http://172.17.0.3:8501/"  # mfcc

tensorflow_server2_url = "http://172.17.0.2:8501/"  # spec 

# Initialize our Flask app.

app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():

    
    # Assume that this function 'retrieve_uploaded_file_name' has already been written for you.
    # It inspects the uploaded files, saves it to disk, and returns the file path to that file to you.
    
    uploaded_file_name = retrieve_uploaded_file_name(request)      
    
    # TODO:
    # Call the convert_mfcc and convert_spec functions here
    # ..................... CODES START HERE ..................... #

    mfcc_file = convert_mfcc(uploaded_file_name) 
    mfcc_file = mfcc_file.reshape(1, mfcc_file.shape[0], mfcc_file.shape[1], 1 ) # reshape mfccc to single record

    spec_file = convert_spectral(uploaded_file_name)
    spec_file = spec_file.reshape(1, spec_file.shape[0], spec_file.shape[1], 1 ) # reshape spectral to single record
    
    # ...................... CODES END HERE ...................... #

    # TODO:
    # Send one or more POST requests to the Tensorflow serving servers. 
    # ..................... CODES START HERE ..................... #

    mfcc_url = tensorflow_server1_url + 'v1/models/mfcc/versions/1:predict'
    spec_url = tensorflow_server2_url + 'v1/models/spec/versions/1:predict'

    headers = {'Content-Type': 'application/json'}

    mfcc_response = requests.post(url = mfcc_url, headers = headers, json = mfcc_file);

    spec_response = requests.post(url = spec_url, headers = headers, json = spec_file);
   
     # If successful mfcc
    if mfcc_response.status_code == 200:

        # Retrieve the response and send it back as-is to
        # the calling application.
        
        mfcc_y =  mfcc_response.json()
        return Response(json.dumps(y), mimetype='application/json') 

    return Response("{}", mimetype='application/json')    

    # If successful spec 
    if spec_response.status_code == 200:

        # Retrieve the response and send it back as-is to
        # the calling application.
        
        spec_y = spec_response.json()
        return Response(json.dumps(y), mimetype='application/json') 

    return Response("{}", mimetype='application/json') 
    # ...................... CODES END HERE ...................... #    

    # TODO:
    # Sum or average the results and send the class number with the highest
    # score back to the caller.
    # ..................... CODES START HERE ..................... #
    
    #ensemble_predict sum the response mfcc_y and spec_y

    mfcc_y = np.array(mfcc_y["predictions"])
    spec_y = np.array(spec_y["predictions"])
    sum = ensemble_predict(mfcc_y, spec_y)
    label_index = np.argmax(sum, axis=1) # highest class number 

    return label_index
    
    # ...................... CODES END HERE ...................... #  